In [9]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [10]:
save_dir=f"{exps_dir}/exp3/exp_gan"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [11]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(11802, 37) (11802, 1) (3300, 37) (3300, 1)


### (2) Lựa chọn các đặc trưng

In [12]:
x_train

,POLICY_NUMBER,INSURANCE_TYPE,PREMIUM_AMOUNT,CLAIM_AMOUNT,CUSTOMER_NAME,ADDRESS_LINE1,ADDRESS_LINE2,CITY,STATE,POSTAL_CODE,MARITAL_STATUS,AGE,TENURE,EMPLOYMENT_STATUS,NO_OF_FAMILY_MEMBERS,RISK_SEGMENTATION,HOUSE_TYPE,SOCIAL_CLASS,ROUTING_NUMBER,CUSTOMER_EDUCATION_LEVEL,INCIDENT_SEVERITY,AUTHORITY_CONTACTED,ANY_INJURY,POLICE_REPORT_AVAILABLE,INCIDENT_STATE,INCIDENT_CITY,INCIDENT_HOUR_OF_THE_DAY,Squared_PREMIUM_AMOUNT,Cubed_PREMIUM_AMOUNT,BoxCox_PREMIUM_AMOUNT,YeoJohnson_PREMIUM_AMOUNT,Quantile_PREMIUM_AMOUNT,Log_CLAIM_AMOUNT,Sqrt_CLAIM_AMOUNT,BoxCox_CLAIM_AMOUNT,YeoJohnson_CLAIM_AMOUNT,Quantile_CLAIM_AMOUNT
0,2013,0,1.510020,-0.002008,1769,2753,2,2119,8,1.474091,1,-0.452465,-0.211687,1,2,1,1,2,1.668448,0,1,2,0,0,18,204,13,1.598790,0.927851,1.167455,1.102467,1.320581,0.729075,0.298801,0.857072,0.636629,0.286951
1,70,1,-0.617961,0.717882,4553,2297,591,1316,1,-0.429244,1,-0.956301,0.862812,1,3,0,0,0,-0.168197,0,2,0,0,0,10,1220,17,-0.785162,-0.779798,-0.789854,-0.762014,-0.650457,1.130793,0.972455,1.376273,1.131485,0.679478
2,8314,4,-0.006981,-0.638526,7596,4948,299,185,42,-0.208714,0,-0.536437,-1.286186,1,2,1,2,2,-0.017970,2,2,0,1,1,39,1366,8,-0.819402,0.343232,-0.290685,1.160576,0.169433,-0.381723,0.913028,-0.116095,0.045910,-0.707896
3,4046,5,-0.258665,-0.541926,234,3612,591,1316,1,-0.428954,1,-1.376164,-0.032604,1,1,2,2,2,1.257583,0,0,2,1,1,18,2097,14,-0.541864,-0.681274,-0.434714,-0.425090,-0.309966,-0.012093,-0.485193,0.008991,-0.171897,-0.254874
4,7710,3,-1.592906,-0.658863,7718,1922,591,1341,56,0.278345,1,-1.544109,-0.868325,1,3,2,0,1,0.940607,2,2,0,0,1,11,52,22,-0.484802,-0.676740,-1.744293,-0.889009,0.488238,0.061397,0.616112,-0.473046,-0.705103,0.406645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11797,8150,3,-0.862753,-0.731347,7851,5336,317,2541,37,-0.121560,1,1.478906,-0.629548,1,4,2,2,2,-0.057968,6,1,2,0,1,23,415,23,0.925738,-0.859254,1.606954,-0.806271,-1.644015,1.209294,-0.680549,-1.624936,1.334730,0.901013
11798,4331,5,0.006804,-0.721898,2548,3254,591,1177,25,-0.295835,1,-1.124246,-1.644352,1,1,1,2,1,0.470626,0,1,0,1,1,18,2097,3,-0.320546,-0.571348,-0.180809,-0.183734,-0.044765,-0.986834,-0.998733,-0.919647,-1.057365,-0.791796
11799,7672,0,-0.659538,-0.478396,7590,3856,591,2601,60,1.551368,1,1.562879,0.206174,1,2,1,1,0,1.083259,0,2,0,1,1,32,1414,1,-0.636481,1.504918,1.340153,0.765041,-2.243509,0.765631,-0.785088,-0.116964,1.852426,-0.361423
11800,8840,5,-0.292461,-0.449240,8979,4558,591,896,39,1.131901,1,-1.292191,-0.480312,1,0,1,0,0,0.719951,2,0,0,1,0,38,395,3,2.652800,0.646472,0.498256,0.084988,0.255474,1.063873,-0.653640,0.117627,0.034512,1.683711


In [13]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [14]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train.CLAIM_STATUS)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo Empty DataFrame
Columns: [Feature, Coefficient]
Index: []
Xếp hạng các đặc trưng theo mức độ quan trọng:
                      Feature  Coefficient
26   INCIDENT_HOUR_OF_THE_DAY    -0.103244
14       NO_OF_FAMILY_MEMBERS    -0.086309
1              INSURANCE_TYPE    -0.072006
21        AUTHORITY_CONTACTED    -0.062546
19   CUSTOMER_EDUCATION_LEVEL    -0.059588
17               SOCIAL_CLASS    -0.051514
15          RISK_SEGMENTATION    -0.049103
20          INCIDENT_SEVERITY    -0.045200
34        BoxCox_CLAIM_AMOUNT    -0.038486
16                 HOUSE_TYPE    -0.038217
13          EMPLOYMENT_STATUS    -0.036021
24             INCIDENT_STATE     0.032611
23    POLICE_REPORT_AVAILABLE    -0.031406
31    Quantile_PREMIUM_AMOUNT    -0.028943
2              PREMIUM_AMOUNT    -0.028328
10             MARITAL_STATUS    -0.025927
22                 ANY_INJURY    -0.025617
36      Quantile_CLAIM_AMOUNT     0.025405
28       Cubed_PREMIUM_AMOUNT     0.024740
27     Squared_PREMIU

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [15]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train.CLAIM_STATUS)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [16]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train.CLAIM_STATUS)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['BoxCox_CLAIM_AMOUNT', 'BoxCox_PREMIUM_AMOUNT', 'Cubed_PREMIUM_AMOUNT', 'EMPLOYMENT_STATUS', 'PREMIUM_AMOUNT', 'Quantile_CLAIM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'Sqrt_CLAIM_AMOUNT', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT']
RFE
Kiểm tra mô hình LDA
['BoxCox_CLAIM_AMOUNT', 'BoxCox_PREMIUM_AMOUNT', 'Cubed_PREMIUM_AMOUNT', 'EMPLOYMENT_STATUS', 'PREMIUM_AMOUNT', 'Quantile_CLAIM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'Sqrt_CLAIM_AMOUNT', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT']
RFE
Kiểm tra mô hình DTC
['ADDRESS_LINE1', 'AGE', 'BoxCox_PREMIUM_AMOUNT', 'CLAIM_AMOUNT', 'CUSTOMER_NAME', 'INCIDENT_CITY', 'INCIDENT_HOUR_OF_THE_DAY', 'POLICY_NUMBER', 'POSTAL_CODE', 'ROUTING_NUMBER']
RFE
Kiểm tra mô hình RD
['BoxCox_CLAIM_AMOUNT', 'BoxCox_PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'Cubed_PREMIUM_AMOUNT', 'INCIDENT_STATE', 'POLICY_NUMBER', 'PREMIUM_AMOUNT', 'Quantile_CLAIM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'STATE']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [In

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [17]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train.CLAIM_STATUS)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], dtype='object')
Lua chon dac trung GNB
Index(['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], dtype='object')
Lua chon dac trung NN
Index(['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], dtype='object')
Lua chon dac trung BG
Index(['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantil

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [18]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], 'GNB': ['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], 'NN': ['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], 'BG': ['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE', 'Cubed_PREMIUM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'BoxCox_CLAIM_AMOUNT', 'Quantile_CLAIM_AMOUNT'], 'NB': ['POLICY_NUMBER', 'PREMIUM_AMOUNT', 'CUSTOMER_NAME', 'ADDRESS_LINE1', 'ADDRESS_LINE2', 'STATE', 'INCIDENT_STATE

In [19]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['BoxCox_CLAIM_AMOUNT',
  'BoxCox_PREMIUM_AMOUNT',
  'Cubed_PREMIUM_AMOUNT',
  'EMPLOYMENT_STATUS',
  'PREMIUM_AMOUNT',
  'Quantile_CLAIM_AMOUNT',
  'Quantile_PREMIUM_AMOUNT',
  'Sqrt_CLAIM_AMOUNT',
  'Squared_PREMIUM_AMOUNT',
  'YeoJohnson_PREMIUM_AMOUNT'],
 'LDA': ['BoxCox_CLAIM_AMOUNT',
  'BoxCox_PREMIUM_AMOUNT',
  'Cubed_PREMIUM_AMOUNT',
  'EMPLOYMENT_STATUS',
  'PREMIUM_AMOUNT',
  'Quantile_CLAIM_AMOUNT',
  'Quantile_PREMIUM_AMOUNT',
  'Sqrt_CLAIM_AMOUNT',
  'Squared_PREMIUM_AMOUNT',
  'YeoJohnson_PREMIUM_AMOUNT'],
 'DTC': ['ADDRESS_LINE1',
  'AGE',
  'CUSTOMER_NAME',
  'INCIDENT_CITY',
  'INCIDENT_HOUR_OF_THE_DAY',
  'POLICY_NUMBER',
  'POSTAL_CODE',
  'PREMIUM_AMOUNT',
  'ROUTING_NUMBER',
  'YeoJohnson_CLAIM_AMOUNT'],
 'RD': ['BoxCox_CLAIM_AMOUNT',
  'CUSTOMER_NAME',
  'Cubed_PREMIUM_AMOUNT',
  'INCIDENT_STATE',
  'POLICY_NUMBER',
  'PREMIUM_AMOUNT',
  'STATE',
  'Sqrt_CLAIM_AMOUNT',
  'Squared_PREMIUM_AMOUNT',
  'YeoJohnson_PREMIUM_AMOUNT'],
 'LGBM': ['ADDRESS_LINE1',
  

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [20]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['BoxCox_CLAIM_AMOUNT', 'BoxCox_PREMIUM_AMOUNT', 'Cubed_PREMIUM_AMOUNT', 'EMPLOYMENT_STATUS', 'PREMIUM_AMOUNT', 'Quantile_CLAIM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'Sqrt_CLAIM_AMOUNT', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT'], 'LDA': ['BoxCox_CLAIM_AMOUNT', 'BoxCox_PREMIUM_AMOUNT', 'Cubed_PREMIUM_AMOUNT', 'EMPLOYMENT_STATUS', 'PREMIUM_AMOUNT', 'Quantile_CLAIM_AMOUNT', 'Quantile_PREMIUM_AMOUNT', 'Sqrt_CLAIM_AMOUNT', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT'], 'DTC': ['ADDRESS_LINE1', 'AGE', 'CUSTOMER_NAME', 'INCIDENT_CITY', 'INCIDENT_HOUR_OF_THE_DAY', 'POLICY_NUMBER', 'POSTAL_CODE', 'PREMIUM_AMOUNT', 'ROUTING_NUMBER', 'YeoJohnson_CLAIM_AMOUNT'], 'RD': ['BoxCox_CLAIM_AMOUNT', 'CUSTOMER_NAME', 'Cubed_PREMIUM_AMOUNT', 'INCIDENT_STATE', 'POLICY_NUMBER', 'PREMIUM_AMOUNT', 'STATE', 'Sqrt_CLAIM_AMOUNT', 'Squared_PREMIUM_AMOUNT', 'YeoJohnson_PREMIUM_AMOUNT'], 'LGBM': ['ADDRESS_LINE1', 'AGE', 'CLAIM_AMOUNT', 'CUSTOMER_NAME', 'INCIDENT_CITY', 'POLICY_NUMBER'